# Tutorial 3


In [2]:
import sys
import numpy as np
import scipy as sp
from scipy import optimize
import matplotlib.pyplot as plt

In [3]:
import sys
def err(string):
    print(string)
    input('Press return to exit')
    sys.exit(0)


## Bisection Method

In [36]:
## module bisection
''' root = bisection(f,x1,x2,switch=0,tol=1.0e-9).
    Finds a root of f(x) = 0 by bisection.
    The root must be bracketed in (x1,x2).
    Setting switch = 1 returns root = None if
    f(x) increases upon bisection.
'''    


def bisection(f,x1,x2,switch=1,tol=1.0e-9):
    import math
    from numpy import sign
    f1 = f(x1)
    if f1 == 0.0: return x1,0
    f2 = f(x2)
    if f2 == 0.0: return x2,0
    if sign(f1) == sign(f2):
        err('[BS]Root is not bracketed')
    n = int(math.ceil(math.log(abs(x2 - x1)/tol)/math.log(2.0)))
    
    for i in range(n):
        x3 = 0.5*(x1 + x2); f3 = f(x3)
        if (switch == 1) and (abs(f3) > abs(f1)) \
                         and (abs(f3) > abs(f2)):
            return None   
        if f3 == 0.0: return x3,i
        if sign(f2)!= sign(f3): x1 = x3; f1 = f3
        else: x2 = x3; f2 = f3
    return (x1 + x2)/2.0,n


## Newton-Raphson method

In [49]:
## module newtonRaphson
''' root = newtonRaphson(f,df,a,b,tol=1.0e-9).
    Finds a root of f(x) = 0 by combining the Newton-Raphson
    method.
    Calls user-supplied functions f(x) and its derivative df(x).   
'''    
def newtonRaphson(f,df,x0,tol=1.0e-9):
    
    x=x0                 
    for i in range(30):
        fx = f(x)
        if fx == 0.0: return x,i
      # Try a Newton-Raphson step    
        dfx = df(x)
        dx = -fx/dfx
        x = x + dx
        if abs(dx) < tol: return x,i
    print('Too many iterations in Newton-Raphson')


## Convergence of Bisection and Newton's Methods

Consider a function $f(x)=x^3-2x^2+x-3$, 

In [38]:
def f(x):
    return x**3 - 2*x**2+x-3
def fprime(x):
    return 3*x**2-4*x+1


import time

t0 = time.time()
rb,niter=bisection(f,0,3)
t1 = time.time()
print(f'Root:{rb:.6f}, Error:{f(rb)}, Iterations:{niter}, Walltime:{t1-t0:.6f}')


Root:2.174559, Error:-1.3857981429055144e-09, Iterations:32, Walltime:0.000140


In [39]:

import time
t0 = time.time()
rn,niter=newtonRaphson(f,fprime,3)
print(rn, f(rn))
t1 = time.time()
print(f'Root:{rn:.6f}, Error:{f(rn)}, Iterations:{niter}, Walltime:{t1-t0:.6f}')

2.17455941029298 -4.440892098500626e-16
Root:2.174559, Error:-4.440892098500626e-16, Iterations:5,Walltime:0.000291


## Improved Newton-Raphson
The Newton-Raphson method assumes that the root to be computed is initially bracketed in $(a,b)$. The midpoint of the bracket is used as the initial guess of the root. The brackets are updated after each iteration. If a Newton-Raphson iteration does not stay within the brackets, it is disregarded and replaced with bisection. Because `newtonRaphson2` uses the function $f(x)$ as well as its derivative, function routines for both (denoted by $f$ and $df$ in the listing) must be provided by the user.

In [40]:
## module newtonRaphson2
''' root = newtonRaphson(f,df,a,b,tol=1.0e-9).
    Finds a root of f(x) = 0 by combining the Newton-Raphson
    method with bisection. The root must be bracketed in (a,b).
    Calls user-supplied functions f(x) and its derivative df(x).   
'''    
def newtonRaphson2(f,df,a,b,tol=1.0e-9):

    from numpy import sign
    
    fa = f(a)
    if fa == 0.0: return a
    fb = f(b)
    if fb == 0.0: return b
    if sign(fa) == sign(fb): err('Root is not bracketed')
    x = 0.5*(a + b)                    
    for i in range(30):
        fx = f(x)
        if fx == 0.0: return x
      # Tighten the brackets on the root 
        if sign(fa) != sign(fx): b = x  
        else: a = x
      # Try a Newton-Raphson step    
        dfx = df(x)
      # If division by zero, push x out of bounds
        try: dx = -fx/dfx
        except ZeroDivisionError: dx = b - a
        x = x + dx
      # If the result is outside the brackets, use bisection  
        if (b - x)*(x - a) < 0.0:  
            dx = 0.5*(b - a)                      
            x = a + dx
      # Check for convergence     
        if abs(dx) < tol*max(abs(b),1.0): return x
    print('Too many iterations in Newton-Raphson')


## Question
Find the smallest positive zero of using `newtonRaphson`,
$$
f(x)=x^4-6.4 x^3+6.45 x^2+20.538 x-31.752
$$
1. Plot the function. 

2. Show that the convergence is linear, rather than quadratic by slightly adjusting the intial guess $x_0$.

3. Show that the convergence can be sped up by changing the iteration formula to 

$$
x_{i+1}=x_i-2 \frac{f\left(x_i\right)}{f^{\prime}\left(x_i\right)}
$$



In [57]:
def f(x): return x**4 - 6.4*x**3 + 6.45*x**2 + 20.538*x - 31.752
def df(x): return 4.0*x**3 - 19.2*x**2 + 12.9*x + 20.538

root,numIter = newtonRaphson(f,df,2)

print(f'Root ={root}')
print(f'Number of iterations ={numIter}')

Root =2.0999999786199406
Number of iterations =22
